In [1]:
import cupy as cp
from pyqcu.cuda import define
from pyqcu.cuda import io
from pyqcu.cuda import qcu
from pyqcu.cuda.set import params, argv, set_ptrs
#############################
params[define._LAT_X_] = 32
params[define._LAT_Y_] = 32
params[define._LAT_Z_] = 32
params[define._LAT_T_] = 32
params[define._LAT_XYZT_] = params[define._LAT_X_] * \
    params[define._LAT_Y_]*params[define._LAT_Z_]*params[define._LAT_T_]
params[define._GRID_X_], params[define._GRID_Y_], params[define._GRID_Z_], params[
    define._GRID_T_] = define.split_into_four_factors(define.size)
params[define._DATA_TYPE_] = define._LAT_C64_
params[define._VERBOSE_] = 0
params[define._SET_PLAN_] = 1
print("Parameters:", params)
argv[define._MASS_] = 0.0
argv = argv.astype(define.dtype_half(params[define._DATA_TYPE_]))
print("Arguments:", argv)
#############################
gauge_filename = f"quda_wilson-bistabcg-gauge_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5"
print("Gauge filename:", gauge_filename)
gauge = io.hdf5_xxxtzyx2grid_xxxtzyx(params, gauge_filename)
fermion_in_filename = gauge_filename.replace("gauge", "fermion-in")
print("Fermion in filename:", fermion_in_filename)
fermion_in = io.hdf5_xxxtzyx2grid_xxxtzyx(params, fermion_in_filename)
fermion_out_filename = gauge_filename.replace("gauge", "fermion-out")
print("Fermion out filename:", fermion_out_filename)
quda_fermion_out = io.hdf5_xxxtzyx2grid_xxxtzyx(params, fermion_out_filename)
#############################
fermion_out = cp.zeros_like(fermion_in)
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
#############################
qcu.applyInitQcu(set_ptrs, params, argv)
for i in range(10):
    qcu.applyWilsonBistabCgQcu(
    fermion_out, fermion_in, gauge, set_ptrs, params)
    print(f"cp.linalg.norm(fermion_out):{cp.linalg.norm(fermion_out)}")
qcu.applyEndQcu(set_ptrs, params)
#############################
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
print("QUDA Fermion out data:", quda_fermion_out.data)
print("QUDA Fermion out shape:", quda_fermion_out.shape)
print("Difference:", cp.linalg.norm(fermion_out -
      quda_fermion_out)/cp.linalg.norm(quda_fermion_out))
#############################

@My Rank:0/1, Local Rank:0@

Parameters: [     32      32      32      32 1048576       1       1       1       1
       0       0       1       0    1000       3       0       1       4
       4       4       8      24       0      42]
Arguments: [0.e+00 1.e-09 1.e-01]
Gauge filename: quda_wilson-bistabcg-gauge_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 16
All Dest Shape: (3, 3, 4, 2, 32, 32, 32, 16)
Dest Shape: (3, 3, 4, 2, 32, 32, 32, 16)
Fermion in filename: quda_wilson-bistabcg-fermion-in_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 16
All Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Fermion out filename: quda_wilson-bistabcg-fermion-out_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Inde

In [6]:
qcu.applyInitQcu(set_ptrs, params, argv)
for i in range(10):
    qcu.applyWilsonDslashQcu(
    fermion_out, fermion_in, gauge, set_ptrs, params)
    print(f"cp.linalg.norm(fermion_out):{cp.linalg.norm(fermion_out)}")
qcu.applyEndQcu(set_ptrs, params)

cp.linalg.norm(fermion_out):277422.46875
cp.linalg.norm(fermion_out):277422.46875
cp.linalg.norm(fermion_out):277422.46875
cp.linalg.norm(fermion_out):277422.46875
cp.linalg.norm(fermion_out):277422.46875
cp.linalg.norm(fermion_out):277422.46875
cp.linalg.norm(fermion_out):277422.46875
cp.linalg.norm(fermion_out):277422.46875
cp.linalg.norm(fermion_out):277422.46875
cp.linalg.norm(fermion_out):277422.46875
lattice set whole time:0.049922913 sec
